In [66]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [70]:
import pickle

In [71]:
with open('test_X.pkl', 'rb') as f:
    test_X = pickle.load(f)
with open('test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)
with open('test_y.pkl', 'rb') as f:
    test_y = pickle.load(f)

In [72]:

builder_train = tfds.folder_dataset.ImageFolder('szd/videos/')
raw_train = builder_train.as_dataset(split="train", shuffle_files=True)
builder_test = tfds.folder_dataset.ImageFolder('szd/videos/')
raw_test = builder_test.as_dataset(split="test", shuffle_files=True)
builder_val = tfds.folder_dataset.ImageFolder('szd/videos/')
raw_val = builder_val.as_dataset(split="val", shuffle_files=True)

In [75]:
IMG_SIZE = 160 # All images will be resized to 160x160

def format_example(pair):
  image, label = pair['image'], pair['label']
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, tf.one_hot(label, 7)

In [76]:
train = raw_train.map(format_example)
validation = raw_val.map(format_example)
test = raw_test.map(format_example)

In [77]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE =1000

In [78]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [79]:
for image_batch, label_batch in train_batches.take(1):
   pass

image_batch.shape

TensorShape([32, 160, 160, 3])

In [80]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.Xception(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [81]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 5, 5, 2048)


In [82]:
base_model.trainable = False

In [84]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

(32, 2048)


In [85]:
prediction_layer = tf.keras.layers.Dense(7)
prediction_batch = prediction_layer(feature_batch_average)

(32, 7)


In [86]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [87]:
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

In [89]:
initial_epochs = 10
validation_steps = 10
loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

10/10 [==============================] - 13s 553ms/step - loss: 2.1226 - accuracy: 0.1375


In [90]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

initial loss: 2.12
initial accuracy: 0.14


In [91]:

history = model.fit(train_batches,
                        epochs=initial_epochs, 
                        validation_data=validation_batches)


Epoch 1/10


2098/2098 [==============================] - 1332s 628ms/step - loss: 0.8484 - accuracy: 0.6839 - val_loss: 1.7740 - val_accuracy: 0.4922
Epoch 2/10
2098/2098 [==============================] - 1381s 658ms/step - loss: 0.6753 - accuracy: 0.7527 - val_loss: 1.9348 - val_accuracy: 0.5109
Epoch 3/10
2098/2098 [==============================] - 1030s 490ms/step - loss: 0.6236 - accuracy: 0.7710 - val_loss: 2.0509 - val_accuracy: 0.4784
Epoch 4/10
2098/2098 [==============================] - 774s 369ms/step - loss: 0.5926 - accuracy: 0.7851 - val_loss: 2.0176 - val_accuracy: 0.5111
Epoch 5/10
2098/2098 [==============================] - 773s 368ms/step - loss: 0.5739 - accuracy: 0.7901 - val_loss: 1.9999 - val_accuracy: 0.5095
Epoch 6/10
2098/2098 [==============================] - 822s 392ms/step - loss: 0.5592 - accuracy: 0.7962 - val_loss: 2.1739 - val_accuracy: 0.4966
Epoch 7/10
2098/2098 [==============================] - 774s 369ms/step - loss: 0.5482 - accuracy: 0.8009 - val_loss: 2.

In [93]:
base_model.trainable = True

In [94]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

Number of layers in the base model:  132


In [95]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [96]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_1 (Dense)             (None, 7)                 14343     
                                                                 
Total params: 20875823 (79.63 MB)
Trainable params: 9492687 (36.21 MB)
Non-trainable params: 11383136 (43.42 MB)
_________________________________________________________________


In [98]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch =  history.epoch[-1],
                         validation_data=validation_batches)

Epoch 10/16


2098/2098 [==============================] - 1184s 562ms/step - loss: 0.3676 - accuracy: 0.8715 - val_loss: 1.7080 - val_accuracy: 0.6148
Epoch 11/16
2098/2098 [==============================] - 1197s 571ms/step - loss: 0.0971 - accuracy: 0.9660 - val_loss: 2.0736 - val_accuracy: 0.5978
Epoch 12/16
2098/2098 [==============================] - 1136s 541ms/step - loss: 0.0498 - accuracy: 0.9837 - val_loss: 2.1654 - val_accuracy: 0.6026
Epoch 13/16
2098/2098 [==============================] - 1125s 536ms/step - loss: 0.0367 - accuracy: 0.9878 - val_loss: 2.5149 - val_accuracy: 0.6046
Epoch 14/16
2098/2098 [==============================] - 1133s 540ms/step - loss: 0.0290 - accuracy: 0.9907 - val_loss: 2.5557 - val_accuracy: 0.5850
Epoch 15/16
2098/2098 [==============================] - 1135s 541ms/step - loss: 0.0250 - accuracy: 0.9916 - val_loss: 2.5734 - val_accuracy: 0.6276
Epoch 16/16
2098/2098 [==============================] - 1170s 557ms/step - loss: 0.0206 - accuracy: 0.9933 - va

In [99]:
model.layers[0].save_weights('xception.h5')